<div dir="rtl" style="width:90%; margin:auto;">
برخی از object ها برای آزاد کردن منابعی مثل <code>open files</code>, <code>locks</code>, <code>operating system handles</code> و <code>unmanaged objects</code>
نیاز به کدهای صریح دارند. <br>
این فرآیند در .NET به اصطلاح <code>disposal</code> گفته می شود و توسط اینترفیس <code>IDisposable</code> پشتیبانی می شود.

<p>
حافظه مدیریت شده که پر شده از اشیائی که دیگر از آنها استفاده نمی شود، باید در جایی آزاد شود. <br>
این عملکرد تحت عنوان <code>garbage collection</code> شناخته میشود، که توسط <code>CLR</code> انجام می شود.

</p>

<p>
فرایند <code>Disposal</code> با فرآیند <code>garbage collection</code> فرق می کند. در <b>disposal</b> صراحتا منابع را آزاد می کنید، در حالی که این کار در <b>garbage collection</b> به صورت اتوماتیک انجام می شود.

</p>

</div>

### IDisposable, Dispose, and Close

In [ ]:
//.NET defines a special interface for types requiring a tear-down method:
public interface IDisposable
{
    void Dispose();
}

C#’s `using` statement provides a syntactic shortcut for calling `Dispose` on objects that implement `IDisposable`,

In [ ]:
using System.IO;

using (FileStream fs = new FileStream ("myFile.txt", FileMode.Open))
{
    // ... Write to the file ...
}

//The compiler converts this to the following:

FileStream fs = new FileStream ("myFile.txt", FileMode.Open);
try
{
    // ... Write to the file ...
}
finally
{
    if (fs != null) ((IDisposable)fs).Dispose();
}

//The finally block ensures that the Dispose method is called even when an exception
//is thrown or the code exits the block early.

In [ ]:
using System.IO;
//from new verion of C#

using FileStream fs = new FileStream ("myFile.txt", FileMode.Open);
// ... Write to the file ...

In [ ]:
//write your own disposable type 

sealed class Demo : IDisposable
{
    public void Dispose()
    {
        // Perform cleanup / tear-down
    }
}

.NET follows a de facto set of rules in its disposal logic.  
- After an object has been `disposed`, it’s beyond redemption. It `cannot be reactivated`, and `calling its methods or properties` (other than Dispose) throws an
`ObjectDisposedException`.

- Calling an object’s `Dispose method` `repeatedly` causes `no error`.
- If disposable object x “owns” disposable object y, x’s `Dispose` method `automatically` calls `y’s Dispose method`


### Close and Stop

Some types define a method called ***Close*** in `addition to Dispose`.   
nearly all cases it’s either of the following  
- Functionally identical to Dispose
- A functional subset of Dispose

An example of the latter is `IDbConnection`: a `Closed connection` can be ***re-Opened***; a ***Disposed connection*** `cannot`.

Some classes define a `Stop` method (e.g., Timer or HttpListener). A ***Stop method*** may release unmanaged resources, like Dispose, but ***unlike Dispose***, it `allows for re-Starting`.

### When to Dispose

A safe rule to follow (in nearly all cases) is `if in doubt, dispose(اگر شک دارید، دور بریزید)`.  
  
There are, however, three scenarios for **not disposing**:

- When **you don’t** `own` the object—for example, when obtaining a shared object via a static field or property
- When an **object’s Dispose** method does something that `you don’t want`
- When an object’s Dispose method is `unnecessary` by design, and disposing that object would `add complexity to your program`

مثالهایی از مورد دوم   
  
|Type |Disposal function | When not to dispose
| ---- | ---- | ----
|`MemoryStream` | Prevents further I/O | When you later need to read/write the stream
|`StreamReader`,<br> `StreamWriter` | Flushes the reader/writer <br> and closes the underlying <br>stream |When you want to keep the underlying stream<br>open (you must then call Flush on a Stream<br>Writer when you’re done)
| `IDbConnection`   | Releases a database<br>connection and clears the<br>connection string | If you need to re-Open it, you should call Close<br>instead of Dispose
| `DbContext (EF Core)` | Prevents further use | When you might have lazily evaluated queries<br>connected to that context

### Clearing Fields in Disposal

In general, you `don’t need` to clear an object’s fields in its ***Dispose*** method. However, it is good practice to `unsubscribe` from `events` that the object has subscribed to internally over its lifetime.

<p dir="rtl">از زنده نگه داشتن ناخواسته شی از چشم GC جلوگیری می کند</p>